# Chapter 2 Working with Text Data

In [1]:
import re
import itertools
from importlib.metadata import version

import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.12.0


## 2.2 Tokenizing text

In [2]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_training_set = f.read()

print(f"Total number of character: {len(raw_training_set)}")
print(raw_training_set[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
preprocessed_training_set = re.split(r'([,.:;?_!"()\']|--|\s)', raw_training_set)
preprocessed_training_set = [item.strip() for item in preprocessed_training_set if item.strip()]
len(preprocessed_training_set)

4690

In [4]:
preprocessed_training_set[:20]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was']

## 2.3 Converting tokens into token IDs

In [5]:
training_set_tokens = sorted(set(preprocessed_training_set))
print(len(training_set_tokens))

1130


In [6]:
training_set_vocabulary = {token: integer for integer, token in enumerate(training_set_tokens)}
for (token, id_) in itertools.islice(training_set_vocabulary.items(), 10):
    print((token, id_))

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)


In [7]:
class SimpleTokenizerV1:

    def __init__(self, vocabulary: dict[str, int]):
        self.token_to_id = vocabulary
        self.id_to_token = {id_: token for token, id_ in vocabulary.items()}

    def encode(self, text: str) -> list[int]:
        preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        tokens = [item.strip() for item in preprocessed_text if item.strip()]
        ids = [self.token_to_id[token] for token in tokens]
        return ids

    def decode(self, ids: list[int]) -> str:
        text = " ".join([self.id_to_token[id_] for id_ in ids])
        # remove extra spaces before punctuation 
        text = re.sub(r'\s+([,.:;?_!"()\']|--|\s)', r'\1', text)  
        return text

In [8]:
tokenizer_v1 = SimpleTokenizerV1(training_set_vocabulary)
text0 = """"It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride."""
ids0 = tokenizer_v1.encode(text0)
print(ids0)
print(tokenizer_v1.decode(ids0))

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## 2.4 Adding special context tokens

In [9]:
# add special tokens
special_tokens = ["<|endoftext|>", "<unk>"]
all_tokens = training_set_tokens + special_tokens
all_vocabulary = {token: id_ for id_, token in enumerate(all_tokens)}
print(len(all_vocabulary))

1132


In [10]:
class SimpleTokenizerV2:

    def __init__(self, vocabulary: dict[str, int]):
        assert "<unk>" in vocabulary, "Vocabulary must contain the <unk> token."
        self.token_to_id = vocabulary
        self.id_to_token = {id_: token for token, id_ in vocabulary.items()}

    def encode(self, text: str) -> list[int]:
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        tokens = [token.strip() for token in preprocessed if token.strip()]
        ids = [
            self.token_to_id.get(token, self.token_to_id["<unk>"]) 
            for token in tokens
        ]
        return ids

    def decode(self, ids: list[int]) -> str:
        text = " ".join([self.id_to_token[id_] for id_ in ids])
        # remove extra spaces before punctuation 
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)  
        return text

In [11]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text3 = " <|endoftext|> ".join((text1, text2))
print(text3)

tokenizer_v2 = SimpleTokenizerV2(all_vocabulary)
ids3 = tokenizer_v2.encode(text3)
print(ids3)

decoded_ids3 = tokenizer_v2.decode(ids3)
print(decoded_ids3)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<unk>, do you like tea? <|endoftext|> In the sunlit terraces of the <unk>.


## 2.5 Byte pair encoding

In [12]:
tokenizer_gpt2 = tiktoken.get_encoding("gpt2")
text4 = (
    "Hello, do you like tea? <|endoftext|> "
    "In the sunlit terraces of the someunknownPlace."
)
ids4 = tokenizer_gpt2.encode(text4, allowed_special={"<|endoftext|>"})
print(ids4)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 617, 34680, 27271, 13]


In [13]:
decoded_ids4 = tokenizer_gpt2.decode(ids4)
print(decoded_ids4)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the someunknownPlace.


In [14]:
unknown_word = "Akwirw ier"
individual_tokens = tokenizer_gpt2.encode(unknown_word, allowed_special={"<|endoftext|>"})
print(individual_tokens)

decoded_individual_tokens = tokenizer_gpt2.decode(individual_tokens)
print(decoded_individual_tokens)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


## 2.6 Data sampling with a sliding window

In [15]:
class GPTDatasetV1(Dataset):

    def __init__(
        self, text: str, tokenizer, window_length: int, stride: int
    ):
        token_ids = tokenizer.encode(text)
        
        self.input_chunks = []
        self.target_chunks = []
        for start in range(0, len(token_ids) - window_length, stride):
            end = start + window_length
            
            input_chunk = token_ids[start:end]
            target_chunk = token_ids[start + 1:end + 1]
            
            self.input_chunks.append(torch.tensor(input_chunk))
            self.target_chunks.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_chunks)

    def __getitem__(self, idx):
        return self.input_chunks[idx], self.target_chunks[idx]


def create_dataloader_v1(
    text: str, window_length=256, stride=128, 
    batch_size=4, shuffle=True, drop_last=True, num_workers=0    
):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, window_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [16]:
dataloader = create_dataloader_v1(
    raw_training_set, window_length=4, stride=1,
    batch_size=1, shuffle=False
)
dataloader_iter = iter(dataloader)

first_batch = next(dataloader_iter)
print(first_batch)

second_batch = next(dataloader_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [17]:
dataloader = create_dataloader_v1(
    raw_training_set, window_length=4, stride=4, 
    batch_size=8, shuffle=False
)
dataloader_iter = iter(dataloader)
inputs, targets = next(dataloader_iter)
print("Inputs:\n", inputs)
print("Targets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.8 Encoding word positions

In [18]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [19]:
window_length = 4
dataloader = create_dataloader_v1(
    raw_training_set, 
    window_length=window_length, stride=window_length, 
    batch_size=8, shuffle=False
)
dataloader_iter = iter(dataloader)
inputs, targets = next(dataloader_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape: torch.Size([8, 4])


In [20]:
token_embeddings = token_embedding_layer(inputs)
print("Token embeddings shape:", token_embeddings.shape)

Token embeddings shape: torch.Size([8, 4, 256])


In [21]:
context_length = window_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [22]:
input_embeddings = token_embeddings + pos_embeddings
print("Input embeddings shape:", input_embeddings.shape)

Input embeddings shape: torch.Size([8, 4, 256])
